# Bermudan Swaption Pricing

Value a European Swaption using the Hull-White, Black-Derman-Toy and Black-Karasinski models. I analyse relationships to test the model pricing.

NOTE SENSITIVE TO EVEN-ODD NUMBER OF TIME STEPS ON TREE - INVESTIGATE

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree
from financepy.models.bk_tree import BKTree
from financepy.models.bdt_tree import BDTTree

## Discount Curve

In [3]:
value_dt = Date(1, 1, 2011)
settle_dt = value_dt

Set up flat discount curve

In [4]:
rate = 0.0625
libor_curve = DiscountCurveFlat(value_dt,
                                  rate,
                                  FrequencyTypes.SEMI_ANNUAL,
                                  DayCountTypes.ACT_365F)

# The Underlying Swap

We begin with the underlying swap.

In [5]:
exercise_dt = settle_dt.add_years(1)
swap_maturity_dt = settle_dt.add_years(4)
swap_fixed_cpn = 0.060
swap_fixed_freq_type = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.ACT_365F
swap_type = SwapTypes.PAY

We can value the forward starting swap.

In [6]:
swap = IborSwap(exercise_dt,
                    swap_maturity_dt,
                    swap_type,
                    swap_fixed_cpn,
                    swap_fixed_freq_type,
                    swapFixedDayCountType)

We can approximate the value using 2.5 as the approximate duration of a 3 year swap

In [7]:
(rate - swap_fixed_cpn) * 2.5 * ONE_MILLION

6250.0000000000055

In [8]:
swapValue = swap.value(value_dt, libor_curve, libor_curve)
print("Swap Value: %9.5f"% swapValue)

Swap Value: 6214.67070


The estimate was not bad !

In [9]:
swap.print_fixed_leg_pv()

START DATE: 01-JAN-2012
MATURITY DATE: 01-JAN-2015
COUPON (%): 6.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_365F

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | RATE | PMNT     | DF     | PV       | CUM_PV   
--------+-------------+----------+------+----------+--------+----------+----------
      1 | 02-JUL-2012 |  1000000 |  6.0 | 30082.19 | 0.9117 | 27427.17 |  27427.17
      2 | 01-JAN-2013 |  1000000 |  6.0 | 30082.19 |  0.884 |  26593.8 |  54020.97
      3 | 01-JUL-2013 |  1000000 |  6.0 | 29753.42 | 0.8575 | 25512.54 |  79533.51
      4 | 01-JAN-2014 |  1000000 |  6.0 | 30246.58 | 0.8313 | 25143.12 | 104676.64
      5 | 01-JUL-2014 |  1000000 |  6.0 | 29753.42 | 0.8063 | 23989.76 | 128666.39
      6 | 01-JAN-2015 |  1000000 |  6.0 | 30246.58 | 0.7817 | 23642.39 | 152308.78


In [10]:
swap.print_float_leg_pv()

START DATE: 01-JAN-2012
MATURITY DATE: 01-JAN-2015
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | IBOR   | PMNT     | DF     | PV       | CUM_PV   
--------+-------------+----------+--------+----------+--------+----------+----------
      1 | 02-APR-2012 |  1000000 | 6.2023 | 15678.07 | 0.9258 | 14515.36 |  14515.36
      2 | 02-JUL-2012 |  1000000 | 6.2018 | 15504.47 | 0.9117 | 14136.06 |  28651.42
      3 | 01-OCT-2012 |  1000000 | 6.2018 |  15332.2 | 0.8979 | 13766.14 |  42417.57
      4 | 01-JAN-2013 |  1000000 | 6.2023 | 15505.78 |  0.884 |  13707.7 |  56125.27
      5 | 01-APR-2013 |  1000000 | 6.2013 | 15503.16 | 0.8707 | 13498.97 |  69624.24
      6 | 01-JUL-2013 |  1000000 | 6.2018 | 15504.47 | 0.8575 | 13294.56 |   82918.8
      7 | 01-OCT-2013 |  1000000 | 6.2023 | 15505.78 | 0.8443 | 13091.02 |  96009.82
      8 | 01-JAN-2014 |  1000000 | 6.2023 | 15505.78 | 0.8313 | 12889.5

# IborSwaption

This is a class for European-style swaptions. It implements a number of model implementations.

In [11]:
european_swaption_pay = IborSwaption(settle_dt,
                                      exercise_dt,
                                      swap_maturity_dt,
                                      SwapTypes.PAY,
                                      swap_fixed_cpn,
                                      swap_fixed_freq_type,
                                      swapFixedDayCountType)

In [12]:
european_swaption_rec = IborSwaption(settle_dt,
                                      exercise_dt,
                                      swap_maturity_dt,
                                      SwapTypes.RECEIVE,
                                      swap_fixed_cpn,
                                      swap_fixed_freq_type,
                                      swapFixedDayCountType)

In [13]:
european_swaption_pay

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 01-JAN-2011
EXERCISE DATE: 01-JAN-2012
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 01-JAN-2015
SWAP NOTIONAL: 1000000
FIXED cpn: 6.0
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.ACT_365F
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.THIRTY_E_360

Set the volatility to zero so that payer is pure intrinsic

### Black's Model

In [14]:
sigma = 0.00001

In [15]:
model = Black(sigma)

In [16]:
european_swaption_pay.value(value_dt, libor_curve, model)

np.float64(6214.670696710779)

Let's check for put-call parity

In [17]:
sigma = 0.20

In [18]:
model = Black(sigma)

In [19]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15732.91222
REC Value: 9518.24152
PAY - REC: 6214.67070


Yes put-call parity is respected.

### BK Model

In [20]:
a = 0.01
sigma = 0.0000001
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [21]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6346.08093
REC Value:   0.00000
PAY - REC: 6346.08093


In [22]:
a = 0.01
sigma = 0.20
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [23]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15746.57475
REC Value: 9400.49421
PAY - REC: 6346.08054


### BDT Model

In [24]:
sigma = 0.0000001
num_time_steps = 200
model = BDTTree(sigma, num_time_steps)

In [25]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6347.28920
REC Value:   0.00000
PAY - REC: 6347.28920


In [26]:
sigma = 0.20
model = BDTTree(sigma, num_time_steps)

In [27]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15990.91739
REC Value: 9639.93743
PAY - REC: 6350.97996


### HW Model

In [28]:
sigma = 0.0000001
num_time_steps = 200
a = 0.01
model = HWTree(sigma, a, num_time_steps)

In [29]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6348.45763
REC Value:   0.00000
PAY - REC: 6348.45763


In [30]:
sigma = 0.20 * rate
num_time_steps = 200
a = 0.01
model = HWTree(sigma, a, num_time_steps)

In [31]:
value_pay = european_swaption_pay.value(value_dt, libor_curve, model)
value_rec = european_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16217.69298
REC Value: 9869.43817
PAY - REC: 6348.25481


# IborBermudanSwaption

I now examine FinBermudanSwaption and use the tree models to value it. But first let me price it on the tree models as a European option.

## Start By Only Allowing European Exercise

In [32]:
europeanBermSwaptionPay = IborBermudanSwaption(settle_dt,
                                                  exercise_dt,
                                                  swap_maturity_dt,
                                                  SwapTypes.PAY,
                                                  FinExerciseTypes.EUROPEAN,
                                                  swap_fixed_cpn,
                                                  swap_fixed_freq_type,
                                                  swapFixedDayCountType)

In [33]:
europeanBermSwaptionRec = IborBermudanSwaption(settle_dt,
                                                  exercise_dt,
                                                  swap_maturity_dt,
                                                  SwapTypes.RECEIVE,
                                                  FinExerciseTypes.EUROPEAN,
                                                  swap_fixed_cpn,
                                                  swap_fixed_freq_type,
                                                  swapFixedDayCountType)

### Black Karasinski Model

Setting the volatility close to zero so we only have intrinsic value.

In [34]:
black_vol = 0.0000001

In [35]:
sigma = black_vol
a = 0.01
num_time_steps = 200
model = BKTree(sigma, a, num_time_steps)

In [36]:
value_pay = europeanBermSwaptionPay.value(value_dt, libor_curve, model)
value_rec = europeanBermSwaptionRec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6313.74545
REC Value:   0.00000
PAY - REC: 6313.74545


In [37]:
europeanBermSwaptionPay.print_swaption_value()

SWAP PV01: 2.538479623216061
CPN TIME:  1.0 FLOW 0.0
CPN TIME:  1.5013698630136987 FLOW 0.030246575342465755
CPN TIME:  2.0027397260273974 FLOW 0.03008219178082192
CPN TIME:  2.4986301369863013 FLOW 0.03008219178082192
CPN TIME:  3.0027397260273974 FLOW 0.029753424657534246
CPN TIME:  3.4986301369863013 FLOW 0.030246575342465755
CPN TIME:  4.002739726027397 FLOW 0.029753424657534246
CALL TIME:  1.0
CALL TIME:  1.5013698630136987
CALL TIME:  2.0027397260273974
CALL TIME:  2.4986301369863013
CALL TIME:  3.0027397260273974
CALL TIME:  3.4986301369863013
CALL TIME:  4.002739726027397


### Black Derman Toy Model

In [38]:
sigma = 0.0000001
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [39]:
value_pay = europeanBermSwaptionPay.value(value_dt, libor_curve, model)
value_rec = europeanBermSwaptionRec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6311.83846
REC Value:   0.00000
PAY - REC: 6311.83846


In [40]:
sigma = 0.20
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [41]:
value_pay = europeanBermSwaptionPay.value(value_dt, libor_curve, model)
value_rec = europeanBermSwaptionRec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 15937.51370
REC Value: 9618.31192
PAY - REC: 6319.20178


### Hull White Model

In [42]:
sigma = 0.0000001
a = 0.01
num_time_steps = 101
model = HWTree(sigma, a, num_time_steps)

In [43]:
value_pay = europeanBermSwaptionPay.value(value_dt, libor_curve, model)
value_rec = europeanBermSwaptionRec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 6314.50332
REC Value:   0.00000
PAY - REC: 6314.50332


In [44]:
sigma = 0.20 * rate
a = 0.01
num_time_steps = 101
model = HWTree(sigma, a, num_time_steps)

In [45]:
value_pay = europeanBermSwaptionPay.value(value_dt, libor_curve, model)
value_rec = europeanBermSwaptionRec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16237.06135
REC Value: 9922.55803
PAY - REC: 6314.50332


# Now allow Multiple Exercise Dates - Bermudan

Now allow exercise on ALL cpn dates PLUS expiry.

In [46]:
bermudan_swaption_pay = IborBermudanSwaption(settle_dt,
                                               exercise_dt,
                                               swap_maturity_dt,
                                               SwapTypes.PAY,
                                               FinExerciseTypes.BERMUDAN,
                                               swap_fixed_cpn,
                                               swap_fixed_freq_type,
                                               swapFixedDayCountType)

In [47]:
bermudan_swaption_rec = IborBermudanSwaption(settle_dt,
                                               exercise_dt,
                                               swap_maturity_dt,
                                               SwapTypes.RECEIVE,
                                               FinExerciseTypes.BERMUDAN,
                                               swap_fixed_cpn,
                                               swap_fixed_freq_type,
                                               swapFixedDayCountType)

### Black Karasinski Model

In [48]:
sigma = 0.20
a = 0.01
num_time_steps = 100
model = BKTree(sigma, a, num_time_steps)

In [49]:
value_pay = bermudan_swaption_pay.value(value_dt, libor_curve, model)
value_rec = bermudan_swaption_rec.value(value_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 19235.85760
REC Value: 13043.39490
PAY - REC: 6192.46270


### Black-Derman-Toy Model

In [50]:
sigma = 0.20
num_time_steps = 100
model = BDTTree(sigma, num_time_steps)

In [51]:
value_pay = bermudan_swaption_pay.value(settle_dt, libor_curve, model)
value_rec = bermudan_swaption_rec.value(settle_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 19500.06870
REC Value: 13318.31315
PAY - REC: 6181.75555


### Hull-White Model

In [52]:
sigma = 0.01
a = 0.01
num_time_steps = 100
model = HWTree(sigma, a)

In [53]:
value_pay = bermudan_swaption_pay.value(settle_dt, libor_curve, model)
value_rec = bermudan_swaption_rec.value(settle_dt, libor_curve, model)
valuePayRec = value_pay - value_rec
print("PAY Value: %9.5f"% value_pay)
print("REC Value: %9.5f"% value_rec)
print("PAY - REC: %9.5f"% valuePayRec)

PAY Value: 16578.58987
REC Value: 10406.29704
PAY - REC: 6172.29283


The swaption is Bermudan so it can only be exercised on cpn payment dates

Copyright (c) 2020 Dominic O'Kane